In [5]:
#Here are the code used to create training and test data for the experiments. 
#Training and test data should be created or recreated before running any of the experiments.
#Script assumes that the database name is IJCAI2022 and that the source table is called BayesianDynamicChecklistLocalDb
#Please also make sure that the table is assigned to the schema called dbo
import pyodbc
import pandas as pd

In [6]:
def deleteTrainTestTables():
    cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=IJCAI2022;Trusted_Connection=yes;')
    cursor = cnxn.cursor()
    cursor.execute("Drop Table BayesianDynamicChecklistLocalDbRandomized"
                   )
    cnxn.commit()
    for ik in range(0,8):
        cursor.execute("Drop Table BayesianDynamicChecklistLocalDbRandomizedTest"+str(ik)
                   )
        cnxn.commit()
        cursor.execute("Drop Table BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik)
                   )
        cnxn.commit()
def createTrainTestTables():
    cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=IJCAI2022;Trusted_Connection=yes;')
    cursor = cnxn.cursor()
    cursor.execute("Select [RowId],[MuncipalityIndex],[IndustrySubgroupCodeIndex],[InspectionId],[InspectionDateId],[ControlPointText],[IndustryMainAreaCode],[IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[IndustrySubgroupCode],[Fylke],[Municipality],[MunicipalityNumber],[NonCompliance],new into BayesianDynamicChecklistLocalDbRandomized FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDb]"+"  "
                      +"left join (select NewID() as new, InspectionId as inspID from [IJCAI2022].[dbo].BayesianDynamicChecklistLocalDb"+"  group by InspectionId) as a on a.inspId=InspectionId"+
                      " where InspectionDateId<20190601 ORDER BY new"
                   )

        
    #commit the transaction
    
    cnxn.commit()
    length=len(pd.read_sql("SELECT  * "+"FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomized"+"]"+" "+
                      " where InspectionDateId<20190601",cnxn))
    for ik in range(0,8):

        cursor.execute("Select [RowId],[MuncipalityIndex],[IndustrySubgroupCodeIndex],[InspectionId],[InspectionDateId],[ControlPointText],[IndustryMainAreaCode],[IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[IndustrySubgroupCode],[Fylke],[Municipality],[MunicipalityNumber],[NonCompliance],new into BayesianDynamicChecklistLocalDbRandomizedTest"+str(ik) 
                           +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomized] "
                          +" where InspectionDateId<20190601 ORDER BY new OFFSET ("+str(round((length/8)*(ik)))+") ROWS FETCH NEXT ("+str(round((length/8))-1)+") ROWS ONLY"
                       )
        cnxn.commit()
        if ik>0 and ik <7:   
            cursor.execute("Select [RowId],[MuncipalityIndex],[IndustrySubgroupCodeIndex],[InspectionId],[InspectionDateId],[ControlPointText],[IndustryMainAreaCode],[IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[IndustrySubgroupCode],[Fylke],[Municipality],[MunicipalityNumber],[NonCompliance],new into BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik) 
                           +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomized] "
                          +" where InspectionDateId<20190601 ORDER BY new OFFSET ("+str(0)+") ROWS FETCH NEXT ("+str(round((length/8)*ik)-1)+") ROWS ONLY"
                       )
            cnxn.commit()
            cursor.execute("Insert into BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik)+" Select [RowId],[MuncipalityIndex],[IndustrySubgroupCodeIndex],[InspectionId],[InspectionDateId],[ControlPointText],[IndustryMainAreaCode],[IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[IndustrySubgroupCode],[Fylke],[Municipality],[MunicipalityNumber],[NonCompliance],new " 
                          + " FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomized] "
                          +" where InspectionDateId<20190601 ORDER BY new OFFSET ("+str(round((length/8)*(ik+1)))+") ROWS FETCH NEXT ("+str(round((length/8)*(7-ik))-1)+") ROWS ONLY"
                       )
            cnxn.commit()
            
        elif ik<1:
            cursor.execute("Select [RowId],[MuncipalityIndex],[IndustrySubgroupCodeIndex],[InspectionId],[InspectionDateId],[ControlPointText],[IndustryMainAreaCode],[IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[IndustrySubgroupCode],[Fylke],[Municipality],[MunicipalityNumber],[NonCompliance],new into BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik) 
                           +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomized] "
                          +" where InspectionDateId<20190601 ORDER BY new OFFSET ("+str(round((length/8)*(ik+1)))+") ROWS FETCH NEXT ("+str(length-round((length/8)*(ik+1))-1)+") ROWS ONLY"
                       )
            cnxn.commit()
        else:
            cursor.execute("Select [RowId],[MuncipalityIndex],[IndustrySubgroupCodeIndex],[InspectionId],[InspectionDateId],[ControlPointText],[IndustryMainAreaCode],[IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[IndustrySubgroupCode],[Fylke],[Municipality],[MunicipalityNumber],[NonCompliance],new into BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik) 
                           +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomized] "
                          +" where InspectionDateId<20190601 ORDER BY new OFFSET ("+str(0)+") ROWS FETCH NEXT ("+str(length-round((length/8))-1)+") ROWS ONLY"
                       )
            cnxn.commit()
            

def deleteTrainTestTablesParameters():
    cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=IJCAI2022;Trusted_Connection=yes;')
    cursor = cnxn.cursor()
    for ik in range(0,8):
        cursor.execute("Drop Table BayesianDynamicChecklistLocalDbRandomizedTestParameters"+str(ik)
                   )
        cnxn.commit()    
def createTrainTestTablesParameters():
    cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=IJCAI2022;Trusted_Connection=yes;')
    cursor = cnxn.cursor()

    for ik in range(0,8):
        traindateid=""
        testdateid=""
        cursor.execute("SELECT  [MuncipalityIndex] as MuncipalityIndex ,[IndustrySubgroupCodeIndex],[IndustryGroupCode],[IndustryAreaCode],[IndustryCode],[Fylke],[MunicipalityNumber],  [InspectionId] as 'InspectionId' ,[IndustryMainAreaCode],ControlPointText,[IndustrySubgroupCode],[Municipality],[NonCompliance]"
    +",round(isNull(SannsynlighetForBruddGittKPogKommune,(1.0/(6.0+isNull(tttg2,0))))*100,0) as ProbabilityForNonComplianceGivenControlPointandMunicipality"
    +",round(isNull(SannsynlighetForBruddGittKPogNaring,(1.0/(6.0+isNull(tttg,0))))*100,0) as ProbabilityForNonComplianceGivenControlPointandIndustrySubgroup"
    +",round(isNull(AntBruddGittKPogNaring,0),0) as NumberOfNonComplianceGivenControlPointandIndustrySubgroup"
    +",round(isNull(AntBruddGittKPogKommune,0),0) as NumberOfNonComplianceGivenControlPointandMunicipality" 
    +",isNull(MunicipalityPar01,1) as MunicipalityPar1"
    +",IIF(SannsynlighetForBruddGittKPogKommune is null,(6.0+isNull(tttg2,0)),isNull(MunicipalityParTot1,6)) as MunicipalityParTot"
    +",isNull(NaringPar1,1) as IndustryPar1"
    +",IIF(SannsynlighetForBruddGittKPogNaring is null,(6+isNull(tttg,0)),isNull(NaringParTot,6))  as IndustryParTot" 
    +" into BayesianDynamicChecklistLocalDbRandomizedTestParameters"+str(ik)
                          +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTest"+str(ik)+"]"
    +" Left join(Select ( (Convert(real,isNull(Count(IndustrySubgroupCode),0))+1.0)/(AVG(Convert(real, Kommune3))+6.0))  as SannsynlighetForBruddGittKPogNaring,Count(IndustrySubgroupCode) as AntBruddGittKPogNaring,ControlPointText as KPT2, IndustrySubgroupCode as Nugk"
    +", (Convert(real,isNull(Count(IndustrySubgroupCode),0))+1.0) as NaringPar1,(AVG(Convert(real, Kommune3))+6.0) as NaringParTot"
                       +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik)+"]"+" "  
    +" join (Select Count(IndustrySubgroupCode) as Kommune3 , ControlPointText as KT2, IndustrySubgroupCode as Kom"
    +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik)+"]"+" "+traindateid+"  group by ControlPointText, IndustrySubgroupCode) as a on a.Kom=IndustrySubgroupCode and ControlPointText=KT2" 
    +" join (Select Count(Municipality) as Kommune5 ,  max(NonCompliance) as AntB2"
    +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik)+"]"+" " +traindateid+") as c on c.AntB2=NonCompliance"
                        +traindateid
    +" group by ControlPointText, NonCompliance, IndustrySubgroupCode) as b on b.KPT2=ControlPointText and b.Nugk=IndustrySubgroupCode"
    +" Left join(Select ( (Convert(real,isNull(Count(Municipality),0))+1.0)/(AVG(Convert(real, Kommune3))+6.0))  as SannsynlighetForBruddGittKPogKommune,Count(Municipality) as AntBruddGittKPogKommune, ControlPointText as KPT, Municipality as KOMM"
    +",(Convert(real,isNull(Count(Municipality),0))+1.0) as MunicipalityPar01,(AVG(Convert(real, Kommune3))+6.0) as MunicipalityParTot1"
    +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik)+"]"+"  join (Select Count(Municipality) as Kommune3 , ControlPointText as KT2, Municipality as Kom"
    +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik)+"]"+" "+traindateid+" group by ControlPointText, Municipality) as a on a.Kom=Municipality and ControlPointText=KT2" 
    +" join (Select Count(Municipality) as Kommune5 ,  max(NonCompliance) as AntB2"
    +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik)+"]"+" " +traindateid+") as c on c.AntB2=NonCompliance"
                        +traindateid+"  group by ControlPointText, NonCompliance, Municipality) as a on a.KOMM=Municipality and a.KPT=ControlPointText"
    +" left join(Select Count(IndustrySubgroupCode) as tttg , ControlPointText as KT10, IndustrySubgroupCode as Kom10"
    +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik)+"]"+" "+traindateid+" group by ControlPointText, IndustrySubgroupCode) as z on z.Kom10=IndustrySubgroupCode and ControlPointText=z.KT10" 
    +" left join(Select Count(Municipality) as tttg2 , ControlPointText as KT10, Municipality as NUG10"
    +" FROM [IJCAI2022].[dbo].[BayesianDynamicChecklistLocalDbRandomizedTrain"+str(ik)+"]"+" "+traindateid+" group by ControlPointText, Municipality) as zz on ControlPointText=zz.KT10 and zz.NUG10=Municipality" 
    +" where InspectionDateId<20190601"
                          +" Order by InspectionDateId " + testdateid)

        cnxn.commit()

In [7]:
#deleteTrainTestTables() #Uncomment whenever random resampling is needed
createTrainTestTables() 
#deleteTrainTestTablesParameters() #Uncomment whenever random resampling is needed
createTrainTestTablesParameters()#Filling test sets with precalculated parameters for CBCBR, calculated from the corresponding training set.